### 1. Objectives

- Quantify the extent of class imbalance (Positive, Negative, Neutral) in the Bangla Sentiment Dataset and its impact on sentiment classification performance.
- Evaluate three lightweight mitigation techniques—Synthetic Minority Oversampling Technique (SMOTE), random undersampling, and weighted loss functions—in improving model accuracy and robustness.
- Assess performance differences across dataset sources (newspapers, social media, blogs) to understand source-specific imbalance effects.

### 2. Hypotheses

- **H1**: Class imbalance in the Bangla Sentiment Dataset reduces sentiment classification performance, particularly for minority classes, as measured by per-class F1-scores.
- **H2**: Mitigation techniques (SMOTE, undersampling, weighted loss) significantly improve model performance compared to baseline models, as measured by weighted F1-score and ROC-AUC.
- **H3**: Mitigation effectiveness varies across sources (newspapers, social media, blogs) due to differences in linguistic style and imbalance patterns.



In [1]:
# Load the dataset
import pandas as pd

# Load the Excel file
df = pd.read_excel('data-source/sentiment_dataset.xlsx', engine='openpyxl')

# Show first 5 rows
print(df.head())

                                               Tense  Label
0  জিনিসপত্রের অতিরিক্ত দাম বৃদ্ধির জন্য এই শহরে ...    0.0
1  সঠিক ভাবে তদারকি করলে এই সমস্যা থেকে পরিত্রান ...    1.0
2  দেশের টাকা যখন বিদেশে চোলে যাচ্ছে তখন দেশের সর...    0.0
3          ওনার মতো ব্যর্থ মন্ত্রীর পদত্যাগ করা উচিত    0.0
4                 আল্লাহ তোদের বিচার করবে অপেক্ষা কর    0.0


In [3]:
# Convert to csv for better load speed
df.to_csv('data-source/data.csv', index=False)


# Import all necessary libraries

In [43]:
# Basic data processing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil
# Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score,
    recall_score, f1_score,
    classification_report, confusion_matrix
)

# MLflow tracking
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

In [47]:
# Load the csv file
df = pd.read_csv('data-source/data.csv', encoding='utf-8')

df.head()

,Tense,Label
0,জিনিসপত্রের অতিরিক্ত দাম বৃদ্ধির জন্য এই শহরে ...,0.0
1,সঠিক ভাবে তদারকি করলে এই সমস্যা থেকে পরিত্রান ...,1.0
2,দেশের টাকা যখন বিদেশে চোলে যাচ্ছে তখন দেশের সর...,0.0
3,ওনার মতো ব্যর্থ মন্ত্রীর পদত্যাগ করা উচিত,0.0
4,আল্লাহ তোদের বিচার করবে অপেক্ষা কর,0.0


## Initial Data Exploration

In [48]:
# shape of dataset
print("Shape of dataset: {}".format(df.shape))

Shape of dataset: (9162, 2)


In [49]:
# Display the column names
print("Column names in the dataset:")
print(df.columns)

Column names in the dataset:
Index(['Tense', 'Label'], dtype='object')


In [50]:
# Display the data types of each column
print("Data types of each column:")
print(df.dtypes)

Data types of each column:
Tense     object
Label    float64
dtype: object


In [15]:
# Display information about the dataset
print("Information about the dataset:")
print(df.info())

Information about the dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9162 entries, 0 to 9161
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Tense   9162 non-null   object 
 1   Label   9154 non-null   float64
dtypes: float64(1), object(1)
memory usage: 143.3+ KB
None


In [18]:
# Check for null values in the dataset
print("Null values in the dataset:")
print(df.isnull().sum())

Null values in the dataset:
Tense    0
Label    8
dtype: int64


In [20]:
# Check for duplicate values in the dataset
print("Duplicate values in the dataset:")
print(df.duplicated().sum())

Duplicate values in the dataset:
1412


In [21]:
# Display summary statistics of the dataset
print("Summary statistics of the dataset:")
print(df.describe())

Summary statistics of the dataset:
             Label
count  9154.000000
mean      0.856565
std       0.860452
min       0.000000
25%       0.000000
50%       1.000000
75%       2.000000
max       2.000000


In [24]:
# Separate categorical and numerical features
categorical_features = df.select_dtypes(include="object").columns
numerical_features = df.select_dtypes(exclude="object").columns
print(categorical_features)
print(numerical_features)

Index(['Tense'], dtype='object')
Index(['Label'], dtype='object')


### Phase 1: Data Exploration and Imbalance Analysis

- **Objective**: Quantify class imbalance and establish baseline characteristics.
- **Steps**:
    1. Load dataset using pandas.
    2. Calculate label distribution (Positive, Negative, Neutral) overall and by source (newspapers, social media, blogs).
    3. Visualize distributions using matplotlib/seaborn (e.g., bar charts, pie charts).
    4. Identify imbalance severity (e.g., ratio of majority to minority classes).

In [32]:
# Load the csv file
df = pd.lo
# Display first few rows to verify
print(df.head())

# Check for missing values
print(df.isnull().sum()) 

                                               Tense  Label
0  জিনিসপত্রের অতিরিক্ত দাম বৃদ্ধির জন্য এই শহরে ...    0.0
1  সঠিক ভাবে তদারকি করলে এই সমস্যা থেকে পরিত্রান ...    1.0
2  দেশের টাকা যখন বিদেশে চোলে যাচ্ছে তখন দেশের সর...    0.0
3          ওনার মতো ব্যর্থ মন্ত্রীর পদত্যাগ করা উচিত    0.0
4                 আল্লাহ তোদের বিচার করবে অপেক্ষা কর    0.0
Tense    0
Label    8
dtype: int64


In [33]:
# Verify unique tense and labels
print("Tense:", df['Tense'].unique())
print("Lables:", df['Label'].unique()) 

Tense: ['জিনিসপত্রের অতিরিক্ত দাম বৃদ্ধির জন্য এই শহরে জীবন ধারণ করা কঠিন হয়ে যাচ্ছে'
 'সঠিক ভাবে তদারকি করলে এই সমস্যা থেকে পরিত্রান পওয়া সম্ভব'
 'দেশের টাকা যখন বিদেশে চোলে যাচ্ছে তখন দেশের সরকার কী করেন ' ...
 'দয়া করে কেও ফুড পান্ডায় ওয়াডার দিবেন না'
 'আপনাদের কাছে টাকা আছে ভালো কথা  আপনারা রেষ্টুরেন্টে গিয়ে পছন্দসই খাবার অর্ডার করে গরম খাবার খাবেন'
 'এই ঘরবন্দি জীবনে ফুডপান্ডার ভুমিকা সত্যিই প্রসংশনীয় ভালো কাজের জন্যে শুভকামনা']
Lables: [ 0.  1.  2. nan]


In [39]:
# Overall label distribution
overall_distribution = df['Label'].value_counts()
overall_distribution_percent = df['Label'].value_counts(normalize=True) * 100
print("Overall Label Distribution (Counts):")
print(overall_distribution)
print("\nOverall Label Distribution (Percentages):")
print(overall_distribution_percent)

Overall Label Distribution (Counts):
Label
0.0    4139
2.0    2826
1.0    2189
Name: count, dtype: int64

Overall Label Distribution (Percentages):
Label
0.0    45.215206
2.0    30.871750
1.0    23.913043
Name: proportion, dtype: float64
